## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-02-26-10 +0000,48hills,Under the Stars: Fresh new music nods to wrap ...,https://48hills.org/2025/12/under-the-stars-fr...
1,2025-12-04-02-17-53 +0000,nypost,"Family of boy, 11, allegedly killed by illegal...",https://nypost.com/2025/12/03/us-news/family-o...
2,2025-12-04-02-14-00 +0000,wsj,How a Man Convicted of Running a Latin America...,https://www.wsj.com/politics/policy/how-a-man-...
3,2025-12-04-02-14-00 +0000,wsj,30-Year JGB Yield Hits Fresh High Ahead of Auc...,https://www.wsj.com/finance/investing/30-year-...
4,2025-12-04-02-03-06 +0000,nyt,Pentagon Watchdog Finds Hegseth’s Signal Chat ...,https://www.nytimes.com/2025/12/03/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
87,trump,53
3,new,17
65,he,14
79,ukraine,12
86,house,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...,109
40,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,99
38,2025-12-04-00-11-47 +0000,nyt,Trump Administration Begins Immigration Operat...,https://www.nytimes.com/2025/12/03/us/border-p...,90
286,2025-12-03-13-18-46 +0000,bbc,Trump says he does not want Somalis in US as I...,https://www.bbc.com/news/articles/c208x9v68w3o...,88
271,2025-12-03-14-11-30 +0000,nypost,‘Grim Reaper’ White House budget chief Russ Vo...,https://nypost.com/2025/12/03/us-news/wh-budge...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,109,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...
45,78,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
281,53,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
52,47,2025-12-03-23-50-09 +0000,missionlocal,"Derrick Lew, 22-year department veteran, named...",https://missionlocal.org/2025/12/derrick-lew-s...
6,45,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
368,44,2025-12-03-04-10-00 +0000,wsj,Republican Matt Van Epps won the special elect...,https://www.wsj.com/politics/elections/republi...
1,44,2025-12-04-02-17-53 +0000,nypost,"Family of boy, 11, allegedly killed by illegal...",https://nypost.com/2025/12/03/us-news/family-o...
208,37,2025-12-03-18-05-36 +0000,nypost,Private employers cut 32K jobs last month — hi...,https://nypost.com/2025/12/03/business/private...
350,36,2025-12-03-07-43-08 +0000,nypost,WestJet airline under fire after agent tears u...,https://nypost.com/2025/12/03/world-news/westj...
138,34,2025-12-03-21-20-09 +0000,nypost,MyPillow CEO Mike Lindell files paperwork to c...,https://nypost.com/2025/12/03/us-news/mypillow...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
